In [2]:
import numpy as np
import torch
from facenet_pytorch import MTCNN, InceptionResnetV1
import cv2
from scipy.spatial.distance import cosine

# Paths
embeddings_path = r"C:\Code\vision-attendance-system\face_embeddings.npy"
test_image_path = r"C:\Code\Alan Guedes - CSMPR21_data\captured_faces\camera_1\01\141526_110523_face_1.png"

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load FaceNet model
model = InceptionResnetV1(pretrained="vggface2").to(device).eval()
mtcnn = MTCNN(keep_all=False, device=device)

# Load stored embeddings
face_embeddings = np.load(embeddings_path, allow_pickle=True).item()

# Read test image
img = cv2.imread(test_image_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Detect face
face = mtcnn(img)
if face is None:
    print("No face detected in the image.")
    exit()

# Extract embeddings
face_embedding = model(face.unsqueeze(0).to(device)).detach().cpu().numpy().flatten()

# Compare with stored embeddings
best_match = None
best_distance = float("inf")

for person_id, embedding in face_embeddings.items():
    distance = cosine(face_embedding, embedding)
    if distance < best_distance:
        best_match = person_id
        best_distance = distance

# Set threshold
THRESHOLD = 0.9

if best_distance < THRESHOLD:
    print(f"Match found: {best_match} (Similarity: {1 - best_distance:.3f})")
else:
    print(f"No match found (Closest: {best_match}, Distance: {best_distance:.3f})")

Match found: Test_Ref_01 (Similarity: 0.395)
